In [ ]:
from Code.training import Trainer
import matplotlib.pyplot as plt
import networkx as nx
import torch
import pandas as pd
import numpy as np
import scipy
from Code.utils import graph_from_scores, plot_scores
%matplotlib inline

In [ ]:
__HOME_DIR__ = "/home/du4/19CS30053/MTP2"
__MAX_NODES__ = 200

In [ ]:
import sys
sys.path.append(f"{__HOME_DIR__}/Model/src/")
from lib import *
from dataset import Heterogeneous_Graph_Dataset

In [ ]:
dataset_name = input("Enter the dataset name:")

In [ ]:
split_criteria = input("Enter the split criteria:")

In [ ]:
split_criteria = (split_criteria).split(',') 
if len(split_criteria)==1:
    split_criteria=split_criteria[0]

In [ ]:
dataset_title = dataset_name
if isinstance(split_criteria, str):
    dataset_title+="_"
    dataset_title+=split_criteria
if isinstance(split_criteria, list):
    dataset_title+="_"
    for s in split_criteria:
        dataset_title+="_"
        dataset_title+=s

dataset_title = "single_" + dataset_title

In [ ]:
dataset = Heterogeneous_Graph_Dataset(dataset_name=dataset_name, split_criteria=split_criteria)
dataset.get_categorical_graph(on_split_data=True)

# All homogeneous graphs
real_data_list = dataset.split_data_category_graph

new_real_data_list = []
index_of_graph_with_max_nodes = -1
num_max_nodes = 0
for i, g in enumerate(real_data_list):
    g.edge_index = g.edge_index.long()
    if (index_of_graph_with_max_nodes==-1 or g['x'].shape[0]>num_max_nodes) and g['x'].shape[0] <= __MAX_NODES__ :
        index_of_graph_with_max_nodes=i
        num_max_nodes = g['x'].shape[0]
new_real_data_list.append(real_data_list[index_of_graph_with_max_nodes])

real_loader = DataLoader(new_real_data_list, batch_size=1024, shuffle=True)

In [ ]:
create_every = 500
plot_every = 1

In [ ]:
for real in tqdm(real_loader):
    adj = (to_dense_adj(real.edge_index, max_num_nodes=real.num_nodes)[0]>0.0).float().cpu().detach().numpy()
    graph_sparse = scipy.sparse.csr_matrix(adj)

    trainer = Trainer(graph_sparse, len(adj), max_iterations=20000, rw_len=12, batch_size=128, H_gen=40, H_disc=30, H_inp=128, z_dim=16, lr=0.0003,
                  n_critic=3, gp_weight=10.0, betas=(.5, .9), l2_penalty_disc=5e-5, l2_penalty_gen=1e-7, temp_start=5.0,  
                  val_share=0.2, test_share=0.1, seed=20, set_ops=False)
    
    trainer.train(create_graph_every=create_every, plot_graph_every=plot_every, num_samples_graph=50000, stopping_criterion='val', folder=f'{dataset_title}/actor_critic_plots')

    plot_scores(trainer, create_every, filepath=f'{dataset_title}/score.png')

    # save model
    torch.save(trainer, f'{dataset_title}/Trainer.pt')